# Class Workbook

## In class activity

In [ ]:
import numpy as np
from scipy.special import expit

import seaborn as sns
import math
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
import pandas as pd
#import statsmodels.api as sm
from plotnine import *
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
import patchworklib as pw
from mpl_toolkits.mplot3d import Axes3D

#import ISLP as islp

### Simulated example

Let's simulate lightbulb life time in minutes using exponential distribution.
Let's say we have A light bulb with rate 0.001 and B light bulb with rate 0.0015.
We will generate 100 bulbs each.

In [ ]:
np.random.seed(1234)

def lightbulb(n=100, hrate=0.001):
    return np.random.exponential(scale=1/hrate, size=n)

# Generate data for columns A and B
df = pd.DataFrame({
    'A': lightbulb(100, 0.001),
    'B': lightbulb(100, 0.0015)
})
df

There are many ways to look at this data.

### 50 hour survival

For example, how many survived 50 hours?

In [ ]:
df_melt=pd.melt(df<3000)
(
 ggplot(df_melt)+geom_bar()+aes(x="variable", fill="value")+xlab("")+coord_flip()
)

### Rate by 10 hour window

Or the counts of how many died in every 10 hour window.

In [ ]:
bin_edges = np.arange(0, 10001, 600)
bin_edges[-1] = 10000  # Ensure the last bin ends at 10000

# Apply the cut function to each column of the DataFrame
dfdisc = df.apply(lambda x: pd.cut(x, bins=bin_edges))

# Melt the DataFrame
mdfdisc = pd.melt(dfdisc)
(
  ggplot(mdfdisc)+geom_bar()+aes(x="value" )+xlab("")+facet_grid("variable~.")
)

### Hours until the lightbulb burns out

Or, we can look at the hours directly.

In [ ]:
(
  ggplot(pd.melt(df))+geom_histogram(bins=30)+
  aes(x="value" )+xlab("")+facet_grid("variable~.")
)

### What is the correct way to analyze?

Depending on the research question the way to analyze the data will be different.

- For the binary problem, you can use logistic regression.
- For the counts you can use Poisson regression with offset.
- For the hours, it depends on the shape of the distribution.

First take away message: It is important is to understand that not all survival analysis requires a complex survival analysis model.  The question then is what requires survival analysis?

### The Veterans' Administration Lung Cancer Trial

Survival in patients with advanced lung cancer from the North Central Cancer Treatment Group. Performance scores rate how well the patient can perform usual daily activities.

- `inst`:	Institution code
- `time`:	Survival time in days
- `status`:	censoring status
  - 1=censored,
  - 2=dead
- `age`:	Age in years
- `sex`:
  - Male=1
  - Female=2
- `ph.ecog`:	ECOG performance score as rated by the physician.
  - 0=asymptomatic,
  - 1= symptomatic but completely ambulatory,
  - 2= in bed <50% of the day,
  - 3= in bed > 50% of the day but not bedbound,
  - 4 = bedbound
- `ph.karno`:	Karnofsky performance score (bad=0-good=100) rated by physician
- `pat.karno`:	Karnofsky performance score as rated by patient
- `meal.cal`:	Calories consumed at meals
- `wt.loss`:	Weight loss in last six months (pounds)

In [ ]:
#pip install scikit-survival
# if you get an error you might need to manually install osqp
#https://osqp.org/docs/get_started/sources.html
from lifelines.datasets import load_lung
lung = load_lung()
lung=lung.dropna()

lung["status"]=np.where(lung["status"] == 1, True, False)
data_y=lung.loc[:,["status","time"]]

data_x=lung.loc[:,["inst","age","sex", "ph.ecog","ph.karno",  "pat.karno",  "meal.cal" , "wt.loss"]]

lung_df=lung.loc[:,["status","time","sex","age"]]
lung_df["sex"]=lung_df["sex"] -1
lung_df.dtypes
lung_df.isnull().sum()

lung_df_m=lung_df[lung.sex==1]
lung_df_f=lung_df[lung.sex==2]

### K-M plot

Kaplan-Meier curves shows the empirical survival probability.  This is the first thing people do when they start survival analysis.  There are few ways to plot the same thing.  One way is using `sksurv`.

In [ ]:
from sksurv.nonparametric import kaplan_meier_estimator

time, survival_prob, conf_int = kaplan_meier_estimator(
    data_y["status"], data_y["time"], conf_type="log-log"
)

plt.step(time, survival_prob, where="post")
plt.fill_between(time, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.ylim(0, 1)
plt.ylabel(r"est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.show()

The alternative is to use `lifelines`.

In [ ]:
from lifelines import KaplanMeierFitter
kmf=KaplanMeierFitter()
kmf.fit(lung_df['time'], event_observed=lung_df['status'], label="kmf.plot_survival_function(show_censors=True, \ncensor_styles={'ms': 6, 'marker': 's'})")
kmf.plot_survival_function(show_censors=True, censor_styles={'ms': 6, 'marker': 's'})

### Checking for proportional hazard assumption.

We check the proportional hazard assumption by plotting log time vs cloglog survival.  If the proportional hazard assumption hold, then the lines should be parallel.

In [ ]:
from lifelines import KaplanMeierFitter

def kmsubset(data):
    kmf = KaplanMeierFitter()
    kmf.fit(data['time'], event_observed=data['status'])
    time_0 = kmf.timeline
    surv_0 = kmf.survival_function_
    cloglog_0 = -np.log(-np.log(surv_0))
    logtime_0 = np.log(time_0)
    aa = {'cloglog': cloglog_0, 'logtime': logtime_0}
    return aa

a=kmsubset(data=lung[lung.sex==1] )
b=kmsubset(data=lung[lung.sex==2])

plt.plot(a['logtime'], a['cloglog'], 'blue', linewidth=2, drawstyle='steps', label='Male')
plt.plot(b['logtime'], b['cloglog'], 'red', linewidth=2, drawstyle='steps', label='Female')

plt.xlabel('logtime')
plt.ylabel('cloglog')
plt.legend()
plt.show()

The line seems fairly parallel, which does not prove the proportional hazard assumption but there is no reason to believe otherwise.

The popular model to fit is the Cox Proportional Hazard model

In [ ]:
from lifelines import CoxPHFitter
cph = CoxPHFitter()
cph.fit(lung_df,"time","status", formula="sex+age")
cph.print_summary()

The fitted coefficient

In [ ]:
cph.plot()
plt.show()

The estimated survival functions align well with the empirical survival function.

In [ ]:
from matplotlib import pyplot as plt
ax = plt.subplot(111)
kmfm = KaplanMeierFitter()
kmfm.fit(lung_df_m['time'], event_observed=lung_df_m['status'], label='male')
kmfm.plot_survival_function(ax=ax)
kmff = KaplanMeierFitter()
kmff.fit(lung_df_f['time'], event_observed=lung_df_f['status'], label='female')
kmff.plot_survival_function(ax=ax)
cph.plot_partial_effects_on_outcome(covariates=["sex"],
                         values=[0,1],ax=ax)
plt.show()

You can plot the estimated curves for different set of predictor values.

In [ ]:
cph.plot_partial_effects_on_outcome(covariates=["sex","age"],
                         values=[[0,40],[0,50],[0,60],
                                  [1,40],[1,50],[1,60]])
plt.show()

### Weibull regression

Following example from Zhang (2016) (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5233524/pdf/atm-04-24-484.pdf  ) we will fit a Weibull regression.

In [ ]:
from lifelines import WeibullAFTFitter
weibull_aft = WeibullAFTFitter()
weibull_aft.fit(lung_df, duration_col='time', event_col='status', formula="sex+age")
weibull_aft.print_summary()

Weibull model seem to fit well.

In [ ]:
ax = plt.subplot(111)
kmfm.plot_survival_function(ax=ax)
kmff.plot_survival_function(ax=ax)
weibull_aft.plot_partial_effects_on_outcome(covariates=["sex"],
                         values=[0,1],ax=ax)
plt.show()

In [ ]:
weibull_aft.plot_partial_effects_on_outcome(covariates=["sex","age"],
                         values=[[0,40],[0,50],[0,60],
                                  [1,40],[1,50],[1,60]])


plt.show()

### Concordance index

The concordance index is a value between 0 and 1 where:

- 0.5 is the expected result from random predictions,
- 1.0 is perfect concordance and,
- 0.0 is perfect anti-concordance (multiply predictions with -1 to get 1.0)

In [ ]:
from lifelines.utils import concordance_index
concordance_index(lung_df['time'], -cph.predict_partial_hazard(lung_df), lung_df['status'])
concordance_index(lung_df['time'], -weibull_aft.predict_expectation(lung_df), lung_df['status'])

### AIC model search

In [ ]:
print(weibull_aft.AIC_, cph.AIC_partial_)
from lifelines.utils import find_best_parametric_model
best_model, best_aic_ = find_best_parametric_model(event_times=lung_df['time'],
                                                      event_observed=lung_df['status'],
                                                 scoring_method="AIC")

from lifelines.plotting import qq_plot
qq_plot(best_model)
plt.show()

### Log-rank test

In [ ]:
from lifelines.statistics import logrank_test
results = logrank_test(lung_df_m['time'], lung_df_f['time'], event_observed_A=lung_df_m['status'], event_observed_B=lung_df_f['status'])
results.print_summary()

## Problem Set

### Brain Tumor

This exercise focuses on the brain tumor data, which is included in the ISLR2 R library.

In [ ]:
BrainCancer=pd.read_csv("https://raw.githubusercontent.com/intro-stat-learning/ISLP/main/ISLP/data/BrainCancer.csv")

(a) Plot the Kaplan-Meier survival curve with ±1 standard error
bands, using the `KaplanMeierFitter()` estimator in the lifelines
package.

In [ ]:
#
#

(b) Draw a bootstrap sample of size $n = 88$ from the pairs ($y_i$, $\delta_i$), and compute the resulting Kaplan-Meier survival curve. Repeat this process B = 200 times. Use the results to obtain an estimate of the standard error of the Kaplan-Meier survival curve at each timepoint. Compare this to the standard errors obtained in (a).

In [ ]:
#
#

(c) Fit a Cox proportional hazards model that uses all of the predictors
to predict survival. Summarize the main findings.

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(d) Stratify the data by the value of $k_i$. (Since only one observation has $k_i=40$, you can group that observation together with the observations that have ki=60.) Plot Kaplan-Meier survival curves for each of the five strata, adjusted for the other predictors.

In [ ]:
#
#

### Censoring

This example makes use of the data in

Observation (Y ) |Censoring Indicator ($\delta$) | Covariate (X)
-----------------|-------------------------------|-------------------
26.5             | 1                             | 0.1
37.2             | 1                             | 11
57.3             | 1                             | -0.3
90.8             | 0                             | 2.8
20.2             | 0                             | 1.8
89.8             | 0                             | 0.4

(a) Create two groups of observations. In Group 1, $X <2$, whereas
in Group 2, $X \geq 2$. Plot the Kaplan-Meier survival curves corresponding
to the two groups. Be sure to label the curves so that
it is clear which curve corresponds to which group. By eye, does
there appear to be a difference between the two groups’ survival
curves?

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(b) Fit Cox’s proportional hazards model, using the group indicator as a covariate. What is the estimated coefficient? Write a sentence providing the interpretation of this coefficient, in terms of the hazard or the instantaneous probability of the event. Is there evidence that the true coefficient value is non-zero?

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

(c) Recall from Section 11.5.2 that in the case of a single binary covariate, the log-rank test statistic should be identical to the score statistic for the Cox model. Conduct a log-rank test to determine whether there is a difference between the survival curves for the two groups. How does the p-value for the log-rank test statistic compare to the p-value for the score statistic for the Cox model from (b)?

In [ ]:
#
#

Your answer:

~~~
Please write your answer in full sentences.


~~~

## Extra Material

### Distributions

In [ ]:
from scipy.stats import expon, weibull_min, gamma

# Define a constant function
def const(x, lam):
    return np.repeat(lam, len(x))

# Set up the plots
fig, axs = plt.subplots(3, 4, figsize=(18, 12))

# Exponential Distribution
x = np.linspace(0, 5, 1000)
lam = 2

# Density plot
axs[0, 0].plot(x, expon.pdf(x, scale=1/lam))
axs[0, 0].set(xlabel='T', ylabel='exp(lambda=2)', title='Density')

# CDF plot
axs[0, 1].plot(x, 1 - expon.cdf(x, scale=1/lam))
axs[0, 1].set(xlabel='T', ylabel='exp(lambda=2)', title='CDF')

# Survival plot
axs[0, 2].plot(x, expon.cdf(x, scale=1/lam))
axs[0, 2].set(xlabel='T', ylabel='exp(lambda=2)', title='Survival')

# Hazard plot
axs[0, 3].plot(x, const(x, lam))
axs[0, 3].set(xlabel='T', ylabel='exp(lambda=2)', title='Hazard')

# Weibull Distribution
shape = 2
scale = 1

# Density plot
axs[1, 0].plot(x, weibull_min.pdf(x, shape, scale=scale))
axs[1, 0].set(xlabel='T', ylabel='Weibull(shape=2,scale=1)', title='Density')

# CDF plot
axs[1, 1].plot(x, 1 - weibull_min.cdf(x, shape, scale=scale))
axs[1, 1].set(xlabel='T', ylabel='Weibull(shape=2,scale=1)', title='CDF')

# Survival plot
axs[1, 2].plot(x, weibull_min.cdf(x, shape, scale=scale))
axs[1, 2].set(xlabel='T', ylabel='Weibull(shape=2,scale=1)', title='Survival')

# Hazard plot
axs[1, 3].plot(x, weibull_min.pdf(x, shape, scale=scale) / weibull_min.cdf(x, shape, scale=scale))
axs[1, 3].set(xlabel='T', ylabel='Weibull(shape=2,scale=1)', title='Hazard')

# Gamma Distribution
shape = 3
rate = 2

# Density plot
axs[2, 0].plot(x, gamma.pdf(x, shape, scale=1/rate))
axs[2, 0].set(xlabel='T', ylabel='gamma(shape=3,rate=2)', title='Density')

# CDF plot
axs[2, 1].plot(x, 1 - gamma.cdf(x, shape, scale=1/rate))
axs[2, 1].set(xlabel='T', ylabel='gamma(shape=3,rate=2)', title='CDF')

# Survival plot
axs[2, 2].plot(x, gamma.cdf(x, shape, scale=1/rate))
axs[2, 2].set(xlabel='T', ylabel='gamma(shape=3,rate=2)', title='Survival')

# Hazard plot
axs[2, 3].plot(x, gamma.pdf(x, shape, scale=1/rate) / gamma.cdf(x, shape, scale=1/rate))
axs[2, 3].set(xlabel='T', ylabel='gamma(shape=3,rate=2)', title='Hazard')

# Adjust layout
plt.tight_layout()
plt.show()